In [84]:
import os
import pandas as pd
from PIL import Image

# Load the train_classes.csv file
csv_file = "C:/Hamoye/planet/planet/train_classes.csv"
df = pd.read_csv(csv_file)

# Process image filenames to match the actual filenames in the directory
train_dir = "C:/Hamoye/planet/planet/train-jpg"
df["image_name"] = df["image_name"].apply(lambda x: f"{x}.jpg")

# Create a list to store the filenames of existing images
existing_images = []

# Check if each image exists in the directory
for filename in df["image_name"]:
    file_path = os.path.join(train_dir, filename)
    if os.path.exists(file_path):
        existing_images.append(filename)

# Create a new DataFrame with only the existing images
df_existing = df[df["image_name"].isin(existing_images)].copy()

# Re-check missing files
missing_files = df_existing[~df_existing["image_name"].isin(existing_images)]["image_name"].tolist()
print("Missing files:", missing_files)
print("Total missing files:", len(missing_files))


Missing files: []
Total missing files: 0


In [85]:
import pandas as pd

# Load the train_classes.csv file
csv_file = "C:/Hamoye/planet/planet/train_classes.csv"
train_df = pd.read_csv(csv_file)

# Process image filenames to match the actual filenames in the directory
train_dir = "C:/Hamoye/planet/planet/train-jpg"
train_df["image_name"] = train_df["image_name"].apply(lambda x: f"{x}.jpg")


In [86]:
#Binarize the labels using MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

# Define the column names for the binary labels
binary_label_names = ['agriculture', 'clear', 'cloudy', 'haze', 'partly_cloudy', 'primary', 'water']

# Combine training and validation data to ensure all classes are considered in the binarization
all_tags = train_df['tags'].tolist() + valid_df['tags'].tolist()

# Initialize MultiLabelBinarizer and fit it on all_tags
mlb = MultiLabelBinarizer()
mlb.fit(all_tags)

# Transform the 'tags' column in both dataframes
train_labels = mlb.transform(train_df['tags'])
valid_labels = mlb.transform(valid_df['tags'])

#Create data generators for loading images
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='image_name',
    y_col='tags',  # Use 'tags' instead of binary_label_names
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',  # Set to 'raw' for multi-label classification
    target_size=(128, 128)
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=train_dir,
    x_col='image_name',
    y_col='tags',  # Use 'tags' instead of binary_label_names
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='raw',  # Set to 'raw' for multi-label classification
    target_size=(128, 128)
)

# Create and compile the model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

# Replace this with your actual model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model


Found 40479 validated image filenames.
Found 5 validated image filenames.


In [87]:
from tensorflow.keras.optimizers import Adam

# Compile the model with the new learning_rate argument
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.0001), metrics=["accuracy"])


In [98]:
import os
import numpy as np
from keras.layers import Dropout


import pandas as pd
from PIL import Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MultiLabelBinarizer

# Load the train_classes.csv file
csv_file = "C:/Hamoye/planet/planet/train_classes.csv"
train_df = pd.read_csv(csv_file)

# Process image filenames to match the actual filenames in the directory
train_dir = "C:/Hamoye/planet/planet/train-jpg"
train_df["image_name"] = train_df["image_name"].apply(lambda x: f"{x}.jpg")

# Set hyperparameters
batch_size = 32
num_classes = len(np.unique(train_df['tags'].str.split()))
num_epochs = 10
learning_rate = 0.001

# Define the model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(128, 128, 3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up data generators with class_mode='multi_output'
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='image_name',
    y_col='tags',
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='multi_output',  # Use 'multi_output' for multi-label classification
    target_size=target_size
)

# Train the model with class_mode='multi_output'
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs
)










TypeError: If class_mode="multi_output", y_col must be a list. Received str.